In [ ]:
import sys
import os
sys.path.append(os.path.abspath("../src"))

from data_manipulation_func import load_epex_data, clean_data
print('Successful import')
df = load_epex_data('../data/spot_prices_2025_raw.csv')
df = clean_data(df)
df.head()

Successful import


,MTU (CET/CEST),Day-ahead Price (EUR/MWh),Date,Hour,Weekday,Week,Weekend
0,2025-01-01 00:00:00,118.46,2025-01-01,0,Wednesday,1,False
1,2025-01-01 01:00:00,129.07,2025-01-01,1,Wednesday,1,False
2,2025-01-01 02:00:00,121.10,2025-01-01,2,Wednesday,1,False
3,2025-01-01 03:00:00,94.28,2025-01-01,3,Wednesday,1,False
4,2025-01-01 04:00:00,63.69,2025-01-01,4,Wednesday,1,False
...,...,...,...,...,...,...,...
3498,2025-05-26 19:00:00,149.17,2025-05-26,19,Monday,22,False
3499,2025-05-26 20:00:00,220.96,2025-05-26,20,Monday,22,False
3500,2025-05-26 21:00:00,167.46,2025-05-26,21,Monday,22,False
3501,2025-05-26 22:00:00,161.80,2025-05-26,22,Monday,22,False
